# 如何创建自定义输出解析器
在某些情况下，您可能需要实现一个自定义的[解析器](/docs/concepts/output_parsers/)，将模型输出结构化处理为特定格式。
实现自定义解析器有两种方法：
1. 在[LCEL](/docs/concepts/lcel/)中使用`RunnableLambda`或`RunnableGenerator`——我们强烈推荐在大多数用例中采用这种方式2. 通过继承自某个用于输出的解析基类——这是较为复杂的实现方式
两种方法之间的差异大多是表面上的，主要体现在触发的回调函数不同（例如 `on_chain_start` 与 `on_parser_start`），以及可运行 lambda 函数与解析器在 LangSmith 等追踪平台中的可视化方式上。

## 可运行的Lambda表达式与生成器
推荐的解析方式是使用**可运行lambda表达式**和**可运行生成器**！
在这里，我们将编写一个简单的解析器，用于反转模型输出的大小写。
例如，如果模型输出："喵"，解析器将生成"m喵"。

In [1]:
from typing import Iterable

from langchain_anthropic.chat_models import ChatAnthropic
from langchain_core.messages import AIMessage, AIMessageChunk

model = ChatAnthropic(model_name="claude-2.1")


def parse(ai_message: AIMessage) -> str:
    """Parse the AI message."""
    return ai_message.content.swapcase()


chain = model | parse
chain.invoke("hello")

'hELLO!'

:::提示
LCEL 在使用 `|` 语法组合时，会自动将函数 `parse` 升级为 `RunnableLambda(parse)`。
如果你不喜欢这样，可以手动导入 `RunnableLambda` 然后运行 `parse = RunnableLambda(parse)`。好的，请提供需要翻译的英文文本，我会将其翻译成中文并保持原有的Markdown格式。

流媒体功能正常吗？

In [5]:
for chunk in chain.stream("tell me about yourself in one sentence"):
    print(chunk, end="|", flush=True)

i'M cLAUDE, AN ai ASSISTANT CREATED BY aNTHROPIC TO BE HELPFUL, HARMLESS, AND HONEST.|

不，它不会，因为解析器在解析输出之前会先聚合输入。
如果我们要实现一个流式解析器，可以让解析器接受一个可迭代的输入对象，并通过生成器逐步产出解析结果。结果一经获取即呈现。

In [11]:
from langchain_core.runnables import RunnableGenerator


def streaming_parse(chunks: Iterable[AIMessageChunk]) -> Iterable[str]:
    for chunk in chunks:
        yield chunk.content.swapcase()


streaming_parse = RunnableGenerator(streaming_parse)

:::重要
请将流式解析器封装在 `RunnableGenerator` 中，因为我们可能会停止通过 `|` 语法自动升级它。好的,我会按照要求将英文翻译成中文,并保持markdown格式一致。以下是不包含```markdown标识的纯内容翻译:

:::

In [12]:
chain = model | streaming_parse
chain.invoke("hello")

'hELLO!'

让我们确认流媒体功能正常运作！

In [13]:
for chunk in chain.stream("tell me about yourself in one sentence"):
    print(chunk, end="|", flush=True)

i|'M| cLAUDE|,| AN| ai| ASSISTANT| CREATED| BY| aN|THROP|IC| TO| BE| HELPFUL|,| HARMLESS|,| AND| HONEST|.|

## 继承解析基类

另一种实现解析器的方法是根据需求继承 `BaseOutputParser`、`BaseGenerationOutputParser` 或其他基础解析器类。
通常情况下，我们**不建议**在大多数应用场景中采用这种方法，因为它会导致代码量增加，却无法带来显著的收益。
最简单的输出解析器类型继承自 `BaseOutputParser` 类，且必须实现以下方法：
* `parse`：接收模型的字符串输出并对其进行解析* （可选）`_type`：标识解析器的名称。
当聊天模型或LLM的输出格式错误时，可以抛出一个`OutputParserException`异常，以表明由于输入错误导致解析失败。使用此异常可以让调用解析器的代码以一致的方式处理这些异常。
:::提示 解析器也是可运行对象！🏃
由于 `BaseOutputParser` 实现了 `Runnable` 接口，通过这种方式创建的任何自定义解析器都将成为有效的 LangChain 可运行对象，并自动获得异步支持、批处理接口、日志记录支持等特性。:::

### 简易解析器

这是一个简单的解析器，可以将布尔值的**字符串**表示（例如 `YES` 或 `NO`）解析并转换为对应的 `boolean` 类型。

In [1]:
from langchain_core.exceptions import OutputParserException
from langchain_core.output_parsers import BaseOutputParser


# The [bool] desribes a parameterization of a generic.
# It's basically indicating what the return type of parse is
# in this case the return type is either True or False
class BooleanOutputParser(BaseOutputParser[bool]):
    """Custom boolean parser."""

    true_val: str = "YES"
    false_val: str = "NO"

    def parse(self, text: str) -> bool:
        cleaned_text = text.strip().upper()
        if cleaned_text not in (self.true_val.upper(), self.false_val.upper()):
            raise OutputParserException(
                f"BooleanOutputParser expected output value to either be "
                f"{self.true_val} or {self.false_val} (case-insensitive). "
                f"Received {cleaned_text}."
            )
        return cleaned_text == self.true_val.upper()

    @property
    def _type(self) -> str:
        return "boolean_output_parser"

In [2]:
parser = BooleanOutputParser()
parser.invoke("YES")

True

In [3]:
try:
    parser.invoke("MEOW")
except Exception as e:
    print(f"Triggered an exception of type: {type(e)}")

Triggered an exception of type: <class 'langchain_core.exceptions.OutputParserException'>


让我们测试改变参数化

In [4]:
parser = BooleanOutputParser(true_val="OKAY")
parser.invoke("OKAY")

True

让我们确认其他 LCEL 方法是否存在

In [5]:
parser.batch(["OKAY", "NO"])

[True, False]

In [6]:
await parser.abatch(["OKAY", "NO"])

[True, False]

In [7]:
from langchain_anthropic.chat_models import ChatAnthropic

anthropic = ChatAnthropic(model_name="claude-2.1")
anthropic.invoke("say OKAY or NO")

AIMessage(content='OKAY')

让我们测试一下我们的解析器是否正常工作！

In [8]:
chain = anthropic | parser
chain.invoke("say OKAY or NO")

True

:::note
注意解析器可以处理来自LLM的输出（字符串）或来自聊天模型的输出（`AIMessage`）！:::

### 解析原始模型输出
有时，模型输出中除了原始文本外，还包含重要的额外元数据。其中一个例子是工具调用（tool calling），其中传递给被调用函数的参数会在单独的属性中返回。如果你需要这种更细粒度的控制，可以改为继承 `BaseGenerationOutputParser` 类。
该类需要一个单独的方法 `parse_result`。该方法接收原始模型输出（例如 `Generation` 或 `ChatGeneration` 的列表）并返回解析后的输出。
支持 `Generation` 和 `ChatGeneration` 两种模式，使得解析器能够同时兼容常规的 LLM 和聊天模型。

In [22]:
from typing import List

from langchain_core.exceptions import OutputParserException
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import BaseGenerationOutputParser
from langchain_core.outputs import ChatGeneration, Generation


class StrInvertCase(BaseGenerationOutputParser[str]):
    """An example parser that inverts the case of the characters in the message.

    This is an example parse shown just for demonstration purposes and to keep
    the example as simple as possible.
    """

    def parse_result(self, result: List[Generation], *, partial: bool = False) -> str:
        """Parse a list of model Generations into a specific format.

        Args:
            result: A list of Generations to be parsed. The Generations are assumed
                to be different candidate outputs for a single model input.
                Many parsers assume that only a single generation is passed it in.
                We will assert for that
            partial: Whether to allow partial results. This is used for parsers
                     that support streaming
        """
        if len(result) != 1:
            raise NotImplementedError(
                "This output parser can only be used with a single generation."
            )
        generation = result[0]
        if not isinstance(generation, ChatGeneration):
            # Say that this one only works with chat generations
            raise OutputParserException(
                "This output parser can only be used with a chat generation."
            )
        return generation.message.content.swapcase()


chain = anthropic | StrInvertCase()

让我们来试试新的解析器！它应该能将模型的输出进行反转。

In [23]:
chain.invoke("Tell me a short sentence about yourself")

'hELLO! mY NAME IS cLAUDE.'